In [2]:
%pylab inline
import pandas as pd
if True:
    matplotlib.use('pgf')
    matplotlib.rcParams.update({
        "font.family":
        "serif",  # use serif/main font for text elements
        "text.usetex": True,  # use inline math for ticks
        "axes.unicode_minus": False,
        "pgf.rcfonts":
        False,  # don't setup fonts from rc parameters
        "pgf.preamble": """
            \\usepackage{times}
            \\usepackage[T1]{fontenc}
            \\usepackage[utf8]{inputenc}
            \\usepackage{bm}
            \\usepackage{amsmath}
            \\usepackage{amssymb}
        """,
        "pgf.texsystem": "pdflatex",
    })

Populating the interactive namespace from numpy and matplotlib


In [25]:
from matplotlib.colors import LinearSegmentedColormap

colors = ["#C8C8FF", "#F0F0F0", "#FFC8C8"]
cmap = LinearSegmentedColormap.from_list("mycmap", colors)

def color(t, best):
    best = max(best, 0.01)
    t = max(t, 0.01)
    ratio = t / best
    assert(ratio >= 1)

    ratio = math.log10(ratio)
    ratio = min(ratio, 3)
    ratio *= 0.33333

    r, g, b, alpha = cmap(ratio)
    assert alpha == 1.0
    return f'{r},{g},{b}'
    
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))
fig, ax = plt.subplots(figsize=(1.3,.15))
ax.imshow(gradient, aspect='auto', cmap=cmap)
ax.get_yaxis().set_visible(False)

def g(x):
    return 256 * math.log10(x) / 3
ax.set_xticks([g(x) for x in [2,3,4,5,6,7,8,9,20,30,40,50,60,70,80,90,200,300,400,500,600,700,800,900]], minor=True)

ax.set_xticks([g(x) for x in [1,10,100,1000]], minor=False)
ax.set_xticklabels([1,10,100,'$\ge$ 1000'], fontsize='small')


# ax.axis('off')
fig.tight_layout()
fig.savefig("colorbar.pdf", bbox_inches='tight')

<ipython-input-25-ec303a798e7e>:35: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()


In [31]:
def make_deep_variance_table():
    rows = pd.read_csv("data_deep.csv")

    # Add colors
    for idx, row in rows.iterrows():
        def best_of(*x):
            return min([float(v.split('+-')[0]) for v in x])
        
        best = best_of(
            row['escher'],
            row['abl1'],
            row['abl2'],
            row['dream'])
        
        for col in ['escher', 'abl1', 'abl2', 'dream']:
            val = float(row[col].split('+-')[0])
            mean, std = map(float, row[col].split('+-'))
            expo = 0

            while mean >= 10.0:
                mean /= 10
                std /= 10
                expo += 1
            while mean < 1.0:
                mean *= 10
                std *= 10
                expo -= 1

            if expo != 0:
                rows.at[idx, col] = '\cbox{%s}{\\displaynumE{%.1f}{%.1f}{%d}}' % (color(val, best), mean, std, expo)
            else:
                rows.at[idx, col] = '\cbox{%s}{\\displaynum{%.1f}{%.1f}}' % (color(val, best), mean, std)

    # Output
    pd.set_option('display.max_colwidth',1000)
    s = rows.to_latex(index=False, escape=False, header=False).split('\n')
    s[0] = '\\begin{tabular}{p{3cm}rrrr}'
    s[1] = '\\toprule'
    s = s[:2] + ['{\\bf Game} & {\\bf ESCHER (Ours)} & {\\bf Ablation 1} & {\\bf Ablation 2} & {\\bf DREAM} \\\\ \\midrule'] + s[2:]

    open("tex/deep_variance.tex", "w").write("\n".join(s))

In [32]:
def make_tabular_variance_table():
    rows = pd.read_csv("data_tabular.csv")

    # Add colors
    for idx, row in rows.iterrows():
        def best_of(*x):
            return min([float(v.split('+-')[0]) for v in x])
        
        best = best_of(
            row['escher'],
            row['abl2'],
            row['dream'],
            row['osmccfr'])
        
        for col in ['escher', 'abl2', 'dream', 'osmccfr']:
            val = float(row[col].split('+-')[0])
            mean, std = map(float, row[col].split('+-'))
            expo = 0

            while mean >= 10.0:
                mean /= 10
                std /= 10
                expo += 1
            while mean < 1.0:
                mean *= 10
                std *= 10
                expo -= 1

            if expo != 0:
                rows.at[idx, col] = '\cbox{%s}{\\displaynumE{%.1f}{%.1f}{%d}}' % (color(val, best), mean, std, expo)
            else:
                rows.at[idx, col] = '\cbox{%s}{\\displaynum{%.1f}{%.1f}}' % (color(val, best), mean, std)

    # Output
    pd.set_option('display.max_colwidth',1000)
    s = rows.to_latex(index=False, escape=False, header=False).split('\n')
    s[0] = '\\begin{tabular}{p{3cm}rrrr}'
    s[1] = '\\toprule'
    s = s[:2] + ['{\\bf Game} & {\\bf ESCHER (Ours)} & {\\bf Ablation 2} & {\\bf DREAM} & {\\bf OS-MCCFR} \\\\ \\midrule'] + s[2:]

    open("tex/tabular_variance.tex", "w").write("\n".join(s))

In [33]:
make_deep_variance_table()
make_tabular_variance_table()